In [1]:
import pandas as pd
import os
import glob
import numpy as np

# Attributes Analysis

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
all_data_path = '/Users/sonynka/HTW/IC/data/aboutyou_paging'
df = pd.read_csv(os.path.join(all_data_path, 'data.csv'), sep=';', encoding='utf-8')

## Tags
Analyse most used tags

In [4]:
df.describe()

,attributes,category,color,img_path,img_url,product_name
count,43403,43403,43403,43403,43403,43403
unique,38011,10,9,41406,39788,22613
top,"Gerader Saum,Rippbündchen,Teilungsnähte,Eingefasster Ausschnitt,Jersey,Rundhals-Ausschnitt,Label Plate,Label Patch/Label Flag,Weicher Griff,Material: 50 % Baumwolle, 50 % Polyester,Elastizität: Leicht elastisch,Ärmellänge: Dreiviertelarm,Länge: Normale Länge,Passform: Normale Passform",shirts,blue,kleider/9f0b31ba3fd1a46054a494e38740595b.jpg,https://cdn.aboutstatic.com/file/8971c47870920cc98d8b6c3dfdd48545,SEIDENSTICKER | Fashion-Bluse 'Schwarze Rose'
freq,16,9735,11471,4,6,64


In [5]:
df['tags'] = df['attributes'].apply(lambda x: x.lower().split(','))

In [6]:
tag_count = {}
def count_tags(tags):
    for tag in tags:
        t = tag.lower()
        if t in tag_count:
            tag_count[t] += 1
        else:
            tag_count[t] = 1
            
a = df['tags'].apply(count_tags)

In [7]:
tag_df = pd.Series(tag_count).reset_index().rename(columns={'index': 'tag', 0: 'count'})
num_imgs = df.shape[0]
tag_df['ratio'] = (pd.to_numeric(tag_df['count']) / num_imgs * 100).round(2)
tag_df = tag_df[tag_df['ratio'] > 1]
tag_df = tag_df.sort_values('ratio', ascending=False)

In [8]:
tag_df['tag'].unique().tolist()

['passform: normale passform',
 'unifarben',
 'länge: normale länge',
 'weicher griff',
 'elastizität: leicht elastisch',
 'elastizität: nicht elastisch',
 'rundhals-ausschnitt',
 'ärmellänge: langarm',
 'ärmellänge: viertelarm',
 'jersey',
 'elastizität: elastisch/stretch',
 'all-over-muster',
 'abgesteppter saum/kante',
 'baumwolle',
 'gerader saum',
 'viskose',
 'ärmellänge: ärmellos',
 'ton-in-ton-nähte',
 'länge: lang/maxi',
 'taillierter schnitt',
 'gerader schnitt',
 'label patch/label flag',
 'seitliche eingrifftaschen',
 'materialart: feinstrick',
 'kragenlos',
 'leibhöhe: mid waist',
 'glatter stoff',
 'obermaterial: 100 % baumwolle',
 'länge: kurz/mini',
 'obermaterial: 100 % polyester',
 ' 5 % elasthan',
 'fließender stoff',
 'rippbündchen',
 'länge: knielang',
 'ärmellänge: dreiviertelarm',
 'blusenshirt',
 'jeans',
 'runder saum',
 'elastischer bund/saum',
 'gürtelschlaufen',
 'v-ausschnitt',
 'lockerer schnitt',
 'passform: lockere passform',
 'gestreift',
 'reißverschlu

## Select useful attributes
Find attributes that are used a lot and can be useful for the application.
Create a special column in the dataframe with each of the useful tags.

In [9]:
def create_tag_column(df, tag_list, column_name):
    """ 
        Create a new column in the given dataframe with the specified column name. 
        The column holds the tag if the tag is in the tag_list otherwise it holds np.nan.
    """
    
    df[column_name] = df['tags'].apply(lambda x: [s for s in x if s in tag_list])
    df[column_name] = [x[0].split(':')[-1].strip() if len(x) > 0 else np.nan for x in df[column_name]]
    
    return df

### Ärmellänge

In [10]:
tag_df[tag_df['tag'].str.startswith('ärmellänge')]

,tag,count,ratio
4932,ärmellänge: langarm,12038,27.74
4933,ärmellänge: viertelarm,8296,19.11
4934,ärmellänge: ärmellos,5735,13.21
4930,ärmellänge: dreiviertelarm,3832,8.83
4931,ärmellänge: halbarm,1314,3.03


In [11]:
aermel_list = tag_df[tag_df['tag'].str.startswith('ärmellänge')]['tag'].tolist()
df = create_tag_column(df, aermel_list, 'ärmellänge')

In [12]:
df['ärmellänge'].unique()

array(['ärmellos', 'halbarm', 'langarm', 'dreiviertelarm', 'viertelarm',
       nan], dtype=object)

### Länge

In [13]:
tag_df[tag_df['tag'].str.startswith('länge')]

,tag,count,ratio
2216,länge: normale länge,21839,50.32
2214,länge: lang/maxi,5424,12.50
2212,länge: kurz/mini,4282,9.87
2211,länge: knielang,3864,8.90
2215,länge: langer schnitt,1729,3.98
2209,länge: 7/8-lang,1120,2.58
2213,länge: kurzer schnitt,805,1.85
2208,länge: 3/4-lang,531,1.22


In [14]:
laenge_list = tag_df[tag_df['tag'].str.startswith('länge')]['tag'].tolist()
df = create_tag_column(df, laenge_list, 'länge')

In [15]:
df['länge'].unique()

array(['lang/maxi', 'knielang', 'kurz/mini', '7/8-lang', '3/4-lang', nan,
       'langer schnitt', 'normale länge', 'kurzer schnitt'], dtype=object)

### Ausschnitt

In [16]:
tag_df[tag_df['tag'].str.contains('ausschnitt')]

,tag,count,ratio
3973,rundhals-ausschnitt,14273,32.88
4361,v-ausschnitt,3582,8.25
1449,eingefasster ausschnitt,1537,3.54
4280,tiefer ausschnitt/dekolleté,944,2.17
4304,u-boot-ausschnitt,778,1.79
3979,rückenausschnitt,771,1.78
4527,weiter ausschnitt,441,1.02


In [17]:
ausschnitt_list = tag_df[tag_df['tag'].str.contains('ausschnitt')]['tag'].tolist()
df = create_tag_column(df, ausschnitt_list, 'ausschnitt')

In [18]:
df['ausschnitt'].unique()

array(['rundhals-ausschnitt', nan, 'v-ausschnitt', 'u-boot-ausschnitt',
       'rückenausschnitt', 'tiefer ausschnitt/dekolleté',
       'eingefasster ausschnitt', 'weiter ausschnitt'], dtype=object)

### Muster

In [19]:
tag_df[(tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze$'))]

,tag,count,ratio
1149,all-over-muster,7374,16.99
1820,gestreift,3387,7.80
1788,geblümt/floral,2967,6.84
4122,spitze,2733,6.30
1802,gepunktet,883,2.03


In [20]:
muster_list = tag_df[(tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze$'))]['tag'].tolist()
df = create_tag_column(df, muster_list, 'muster')

In [21]:
df['muster'].unique()

array(['all-over-muster', nan, 'spitze', 'geblümt/floral', 'gepunktet',
       'gestreift'], dtype=object)

### Passform

In [22]:
tag_df[(tag_df['tag'].str.contains('passform'))]

,tag,count,ratio
3863,passform: normale passform,29153,67.17
3861,passform: lockere passform,3505,8.08
3858,passform: figurbetonte passform,2714,6.25
3866,passform: skinny,1936,4.46
3864,passform: regular,1749,4.03
3867,passform: slimfit,1267,2.92
3865,passform: schmale passform,1011,2.33
3862,passform: loosefit,602,1.39


In [23]:
passform_list = tag_df[(tag_df['tag'].str.contains('passform'))]['tag'].tolist()
df = create_tag_column(df, passform_list, 'passform')

In [24]:
df['passform'].unique()

array(['normale passform', 'lockere passform', 'schmale passform',
       'figurbetonte passform', nan, 'regular', 'skinny', 'slimfit',
       'loosefit'], dtype=object)

In [25]:
df = df.drop(['attributes', 'tags', 'product_name'], axis=1)
df.head()

,category,color,img_path,img_url,ärmellänge,länge,ausschnitt,muster,passform
0,kleider,black,kleider/b2e9d38cbcabc6c0e797c7bcc3270262.jpg,https://cdn.aboutstatic.com/file/b2e9d38cbcabc6c0e797c7bcc3270262,ärmellos,lang/maxi,rundhals-ausschnitt,all-over-muster,normale passform
1,kleider,black,kleider/e008ac2d5e5193e29700f280fc82a244.jpg,https://cdn.aboutstatic.com/file/e008ac2d5e5193e29700f280fc82a244,halbarm,knielang,rundhals-ausschnitt,NaN,lockere passform
2,kleider,black,kleider/2566a0c32ed54761c0f2f6d32b2f3e87.jpg,https://cdn.aboutstatic.com/file/2566a0c32ed54761c0f2f6d32b2f3e87,langarm,knielang,NaN,all-over-muster,normale passform
3,kleider,black,kleider/7a48d05c3dae12657b82e6ef45a6959f.jpg,https://cdn.aboutstatic.com/file/7a48d05c3dae12657b82e6ef45a6959f,dreiviertelarm,knielang,v-ausschnitt,all-over-muster,normale passform
4,kleider,black,kleider/ff93ce38fcf715c7fafbf98865a4e3f5.jpg,https://cdn.aboutstatic.com/file/ff93ce38fcf715c7fafbf98865a4e3f5,viertelarm,kurz/mini,rundhals-ausschnitt,NaN,normale passform


# Create dummies

In [26]:
df = df.set_index('img_url')
df.head()

,category,color,img_path,ärmellänge,länge,ausschnitt,muster,passform
img_url,,,,,,,,
https://cdn.aboutstatic.com/file/b2e9d38cbcabc6c0e797c7bcc3270262,kleider,black,kleider/b2e9d38cbcabc6c0e797c7bcc3270262.jpg,ärmellos,lang/maxi,rundhals-ausschnitt,all-over-muster,normale passform
https://cdn.aboutstatic.com/file/e008ac2d5e5193e29700f280fc82a244,kleider,black,kleider/e008ac2d5e5193e29700f280fc82a244.jpg,halbarm,knielang,rundhals-ausschnitt,NaN,lockere passform
https://cdn.aboutstatic.com/file/2566a0c32ed54761c0f2f6d32b2f3e87,kleider,black,kleider/2566a0c32ed54761c0f2f6d32b2f3e87.jpg,langarm,knielang,NaN,all-over-muster,normale passform
https://cdn.aboutstatic.com/file/7a48d05c3dae12657b82e6ef45a6959f,kleider,black,kleider/7a48d05c3dae12657b82e6ef45a6959f.jpg,dreiviertelarm,knielang,v-ausschnitt,all-over-muster,normale passform
https://cdn.aboutstatic.com/file/ff93ce38fcf715c7fafbf98865a4e3f5,kleider,black,kleider/ff93ce38fcf715c7fafbf98865a4e3f5.jpg,viertelarm,kurz/mini,rundhals-ausschnitt,NaN,normale passform


In [27]:
df_dum = pd.get_dummies(df[['category', 'color', 'ärmellänge', 'länge', 'muster', 'passform', 'ausschnitt']]).reset_index()

In [28]:
df_dum.head()

,img_url,category_blusen-und-tuniken,category_hosen,category_jacken,category_jeans,category_jumpsuits-und-overalls,category_kleider,category_roecke,category_shirts,category_strick,...,passform_schmale passform,passform_skinny,passform_slimfit,ausschnitt_eingefasster ausschnitt,ausschnitt_rundhals-ausschnitt,ausschnitt_rückenausschnitt,ausschnitt_tiefer ausschnitt/dekolleté,ausschnitt_u-boot-ausschnitt,ausschnitt_v-ausschnitt,ausschnitt_weiter ausschnitt
0,https://cdn.aboutstatic.com/file/b2e9d38cbcabc6c0e797c7bcc3270262,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,https://cdn.aboutstatic.com/file/e008ac2d5e5193e29700f280fc82a244,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,https://cdn.aboutstatic.com/file/2566a0c32ed54761c0f2f6d32b2f3e87,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,https://cdn.aboutstatic.com/file/7a48d05c3dae12657b82e6ef45a6959f,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,https://cdn.aboutstatic.com/file/ff93ce38fcf715c7fafbf98865a4e3f5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [29]:
df_dum = df_dum.groupby(['img_url']).max().reset_index()

In [30]:
df_dum.head()

,img_url,category_blusen-und-tuniken,category_hosen,category_jacken,category_jeans,category_jumpsuits-und-overalls,category_kleider,category_roecke,category_shirts,category_strick,...,passform_schmale passform,passform_skinny,passform_slimfit,ausschnitt_eingefasster ausschnitt,ausschnitt_rundhals-ausschnitt,ausschnitt_rückenausschnitt,ausschnitt_tiefer ausschnitt/dekolleté,ausschnitt_u-boot-ausschnitt,ausschnitt_v-ausschnitt,ausschnitt_weiter ausschnitt
0,https://cdn.aboutstatic.com/file/00005b236646a41122e2af6608f9bc74,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://cdn.aboutstatic.com/file/000319a4556c44335bd4a97c8ebaa715,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,https://cdn.aboutstatic.com/file/00036d763919b77400505f6534b7f353,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,https://cdn.aboutstatic.com/file/00058a0b83bd94b4d4b5f12e12b239fe,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,https://cdn.aboutstatic.com/file/000610af30d05925ac9cfd0c28c763d6,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [31]:
df_paths = df.groupby(['img_url']).first()['img_path'].reset_index()

In [32]:
df_paths.head()

,img_url,img_path
0,https://cdn.aboutstatic.com/file/00005b236646a41122e2af6608f9bc74,hosen/00005b236646a41122e2af6608f9bc74.jpg
1,https://cdn.aboutstatic.com/file/000319a4556c44335bd4a97c8ebaa715,kleider/000319a4556c44335bd4a97c8ebaa715.jpg
2,https://cdn.aboutstatic.com/file/00036d763919b77400505f6534b7f353,jeans/00036d763919b77400505f6534b7f353.jpg
3,https://cdn.aboutstatic.com/file/00058a0b83bd94b4d4b5f12e12b239fe,hosen/00058a0b83bd94b4d4b5f12e12b239fe.jpg
4,https://cdn.aboutstatic.com/file/000610af30d05925ac9cfd0c28c763d6,strick/000610af30d05925ac9cfd0c28c763d6.jpg


In [33]:
df_dum = df_paths.merge(df_dum, how='inner').drop(['img_url'], axis=1)

In [34]:
df_dum.head()

,img_path,category_blusen-und-tuniken,category_hosen,category_jacken,category_jeans,category_jumpsuits-und-overalls,category_kleider,category_roecke,category_shirts,category_strick,...,passform_schmale passform,passform_skinny,passform_slimfit,ausschnitt_eingefasster ausschnitt,ausschnitt_rundhals-ausschnitt,ausschnitt_rückenausschnitt,ausschnitt_tiefer ausschnitt/dekolleté,ausschnitt_u-boot-ausschnitt,ausschnitt_v-ausschnitt,ausschnitt_weiter ausschnitt
0,hosen/00005b236646a41122e2af6608f9bc74.jpg,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,kleider/000319a4556c44335bd4a97c8ebaa715.jpg,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,jeans/00036d763919b77400505f6534b7f353.jpg,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,hosen/00058a0b83bd94b4d4b5f12e12b239fe.jpg,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,strick/000610af30d05925ac9cfd0c28c763d6.jpg,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [35]:
df_dum.to_csv('/Users/sonynka/HTW/IC/data/aboutyou_paging/img_attr.csv', sep='\t', encoding='utf-8', index=False)